In [1]:
!pip install --upgrade openai --progress-bar off
!pip install -Uqqq datasets --progress-bar off

  Using cached typing_extensions-4.10.0-py3-none-any.whl (33 kB)
  Using cached exceptiongroup-1.2.0-py3-none-any.whl (16 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
  Attempting uninstall: anyio
    Found existing installation: anyio 2.2.0
    Uninstalling anyio-2.2.0:
      Successfully uninstalled anyio-2.2.0
  Attempting uninstall: openai
    Found existing installation: openai 0.26.5
    Uninstalling openai-0.26.5:
      Successfully uninstalled openai-0.26.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rich 6.2.0 requires typing-extensions<4.0.0,>=3.7.4, but you have typing-extensions 4.10.0 which is incompatible.


In [12]:
import pandas as pd
import numpy as np
import openai
import time
import random
from random import randrange
from tenacity import retry, stop_after_attempt, wait_random_exponential, retry_if_exception_type
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

# # To read and write data files in Google Drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

In [3]:
#### Input and output data file names ####
#path = "/content/drive/MyDrive/"
input_data_filename = "signalmedia-1m.jsonl.gz"
preprocessed_data_filename = "signalmedia_news_dataset_sample.csv"
processed_data_filename = "signalmedia_news_dataset_sample_classified.csv"
output_data_json_filename = "news_classification.json"
output_data_csv_filename = "news_classification.csv"

#### OpenAI model ####
model_name = "gpt-3.5-turbo"

In [4]:
# Reading zipped JSONL data as a Pandas DataFrame
raw_news_df = pd.read_json(f"{input_data_filename}", lines = True)

In [21]:
# new_news = pd.read_json("News_Category_Dataset_v3.json",lines=True)
# new_news.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [23]:

# Selecting "News" records
raw_news_df2 = raw_news_df[raw_news_df['media-type'] == "News"]
# Shuffling the dataset
raw_news_df3 = raw_news_df2.sample(frac = 1)
# Selecting top 1000 records/news articles
raw_news_df4 = raw_news_df3.head(1000)
# Saving the preprocessed data as a CSV file
raw_news_df4.to_csv(f"{preprocessed_data_filename}", index = False)

KeyboardInterrupt: 

In [6]:
# Loading the preprocessed data as a Pandas DataFrame
prep_news_df = pd.read_csv(f"{preprocessed_data_filename}")

In [24]:
display(prep_news_df)

,id,content,title,media-type,source,published
0,fa7a8dc2-061f-44bd-8e79-9ba114b4cf4f,The Democratic Alliance (DA) on Thursday reque...,'Motsoeneng must account for lying to Parliament',News,Sowetan,2015-09-25T05:10:12Z
1,0caf3347-e7ba-40ce-9002-dcaf2160f4f0,"By Bernadette Carreon AFP\n\nPORT MORESBY, Pap...",Pacific summit to issue climate message to world,News,China Post,2015-09-06T16:37:27Z
2,e44004b7-4cf0-4f74-9b36-b51fc049e2d5,SOURCE Wounded Warrior Project\n\nWWP Alumnus ...,"Warfighter Outfitters, Inc. Is Latest Wounded ...",News,WTRF,2015-09-10T12:40:37Z
3,6b3aa028-e107-45b7-bfdb-79d6eabeb3ac,Italian broadband operator Tiscali has launche...,Tiscali unveils new cloud portal for Italian b...,News,Telecompaper,2015-09-22T14:10:15Z
4,56e54d49-c7b4-460e-ba31-70e04f3718ec,The board of directors of ServisFirst Bancshar...,ServisFirst Bancshares announces dividend of U...,News,Bloomberg,2015-09-24T08:29:37Z
...,...,...,...,...,...,...
995,a07a736b-1886-428d-9add-3a1ff52b0727,22/09/2015 14:22\r \r (Infocast News) Ma...,Macquarie upgrades Sands China (01928) to Neutral,News,InfocastFN.com,2015-09-22T06:54:51Z
996,e06b0e72-4a15-4267-ab79-97b56b8717e5,– Morocco stocks were lower after the close on...,Morocco stocks lower at close of trade; Morocc...,News,Investing.com,2015-09-28T16:05:12Z
997,e5d1dd37-0795-4a70-9bb6-e7c25dca95ad,Auja — The wealth declaration of Nigeria's new...,"Nigerian president declares wealth, showing 'a...",News,News24,2015-09-04T11:46:39Z
998,54d6bcea-fb51-49d7-9052-724994b04420,A green energy company found to have made mill...,Solar panels firm Helms fined £200k for nuisan...,News,Shropshire Star,2015-09-30T07:42:34Z


In [67]:
prep_news_df.iloc[0]['content']

'The Democratic Alliance (DA) on Thursday requested that SABC chief operating officer Hlaudi Motsoeneng be summoned to appear before the Communications Portfolio Committee for lying to Parliament about the SABC’s finances.  \n“The problem for Motsoeneng is that lying to Parliament constitutes fraud and‚ as such‚ is a criminal offence‚” Gavin Davis‚ DA shadow minister of communications‚ said. \n\nThe SABC annual report tabled in Parliament on Wednesday indicated that the SABC had suffered a R395-million loss in the 2014/15 financial year. \n\n“This contradicts Motsoeneng’s statement before the Communications Portfolio Committee on 22 April 2015. He said‚ ‘The SABC‚ financially‚ we are sustainable.’ \n\n“(Communications) Minister (Faith) Muthambi agreed. ‘The SABC is on a sound financial footing‚’ the minister of communications told the committee. \n\n“Significantly‚ neither Muthambi nor Motsoeneng mentioned the SABC’s projected loss for this financial year. This was despite the existenc

In [25]:
# Defining bot behavior and instructing
SYSTEM_PROMPT = """You are ChatGPT, an intelligent bot. I will give you a news article. You have to classify the news into one of the 43 categories."""

USER_PROMPT_1 = """Are you clear about your role?"""

ASSISTANT_PROMPT_1 = """Sure, I'm ready to help you with your news classification task. Please provide me with the necessary information to get started."""

# Few Shot Prompting
PROMPT = (
"""
Categories:

U.S. NEWS
COMEDY
PARENTING
WORLD NEWS
CULTURE & ARTS
TECH
SPORTS
ENTERTAINMENT
POLITICS
WEIRD NEWS
ENVIRONMENT
EDUCATION
CRIME
SCIENCE
WELLNESS
BUSINESS
STYLE & BEAUTY
FOOD & DRINK
MEDIA
QUEER VOICES
HOME & LIVING
WOMEN
BLACK VOICES
TRAVEL
MONEY
RELIGION
LATINO VOICES
IMPACT
WEDDINGS
COLLEGE
PARENTS
ARTS & CULTURE
STYLE
GREEN
TASTE
HEALTHY LIVING
THE WORLDPOST
GOOD NEWS
WORLDPOST
FIFTY
ARTS
DIVORCE
ESG

If you don't know the category, response "OTHERS".

Output Format:
Category name

Examples:
1. News: New Product Gives Marketers Access to Real Keywords, Conversions and Results Along With 13 Months of Historical Data

SAN FRANCISCO, CA -- (Marketwired) -- 09/17/15 -- Jumpshot, a marketing analytics company that uses distinctive data sources to paint a complete picture of the online customer journey, today announced the launch of Jumpshot Elite, giving marketers insight into what their customers are doing the 99% of the time they're not on your site. For years, marketers have been unable to see what organic and paid search terms users were entering, much less tie those searches to purchases. Jumpshot not only injects that user search visibility back into the market, but also makes it possible to tie those keywords to conversions -- for any web site.

"Ever since search engines encrypted search results, marketers have been in the dark about keywords, impacting not only the insight into their own search investments, but also their ability to unearth high converting keywords for their competitors," said Deren Baker, CEO of Jumpshot. "Our platform eliminates the hacks, assumptions, and guesswork that marketers are doing now and provides real data: actual searches tied to actual conversions conducted by real people with nothing inferred."

Unlike other keyword research tools that receive data through the Adwords API or send bots to cobble together various data inputs and implied metrics, Jumpshot leverages its panel of over 115 million global consumers to analyze real search activity. As a result, Jumpshot is able to provide companies with actionable data to improve the ROI of their search marketing campaigns, SEO tactics and content marketing initiatives.

Available today, Jumpshot Elite provides 13 months of backward-looking data as well as:

Access to real queries used by searchers

Paid and organic results for any website

Visibility into organic keywords, eliminating the "not provided" outcome in web analytics

Real user queries, clicks and transactions instead of machine-generated clicks with inferred results

Ability to tie keywords to real transactions on any website

Variable attribution models and lookback windows

Launched in January, 2015, Jumpshot grew out of the ambitions of a group of smart marketers and data scientists who were frustrated about the limitations of the data they had access to, and excited about the opportunity to provide new insights into online behavior.

The company uses distinctive data sources to paint a complete picture of the online world for businesses, from where customers spend time online to what they do there and how they get from place to place. By tracking the online customer journey down to each click, Jumpshot reveals how and why customers arrive at purchase decisions. The company tracks more data in more detail than other services, tracking 160 billion monthly clicks generated by its extensive data panel.

About Jumpshot

Jumpshot is a marketing analytics platform that reveals the entire customer journey -- from the key sources of traffic to a site, to browsing and buying behavior on any domain. With a panel of 115 million users, Jumpshot provides marketers with the insight to understand what their customers are doing the 99% of the time they're not on their own site -- a scope of information never before attainable. Jumpshot was founded in 2015 and is headquartered in San Francisco.

For more information, please visit www.jumpshot.com.

Image Available: http://www2.marketwire.com/mw/frame_mw?attachid=2889222

Kelly Mayes

The Bulleit Group

615-200-8845

Published Sep. 17, 2015

Copyright © 2015 SYS-CON Media, Inc. — All Rights Reserved.

Syndicated stories and blog feeds, all rights reserved by the author.

Output: TECHNOLOGY

2. News: SOURCE Harwood Feffer LLP

NEW YORK

On July 21, 2015

On this news, VASCO stock nearly 33% and has not recovered.

Our investigation concerns whether the Company board of directors has breached its fiduciary duties to shareholders, grossly mismanaged the Company, and/or committed abuses of control in connection with the foregoing.

If you own VASCO shares and wish to discuss this matter with us, or have any questions concerning your rights and interests with regard to this matter, please contact:

Robert I. Harwood, Esq.

Harwood Feffer

The law firm responsible for this advertisement is Harwood Feffer LLP (www.hfesq.com). Prior results do not guarantee or predict a similar outcome with respect to any future matter.

Logo - http://photos.prnewswire.com/prnh/20120215/MM54604LOGO

To view the original version on PR Newswire, visit:http://www.prnewswire.com/news-releases/harwood-feffer-llp-announces-investigation-of-vasco-data-security-international-inc-300149371.html

©2015 PR Newswire. All Rights Reserved.

Output: BUSINESS

3. {}
Output:
"""
)

In [56]:
from openai import OpenAI
import os 


# Function to pass model name and user prompts and receive response
def openai_chat_completion_response(USER_PROMPT_2):
  client  = OpenAI(api_key='')
  response = client.chat.completions.create(
              model = model_name,
              messages = [
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": USER_PROMPT_1},
                    {"role": "assistant", "content": ASSISTANT_PROMPT_1},
                    {"role": "user", "content": USER_PROMPT_2}
                ]
            )

  return response.choices[0].message.content.strip(" \n")

In [57]:
# Function to classify news articles
def predict_news_category(news_body):
  # Add news article to the prompt
  NEWS = news_body
  FINAL_PROMPT = PROMPT.format(NEWS)
  # Send prompt for inference
  classify_news = openai_chat_completion_response(FINAL_PROMPT)
  time.sleep(20)
  return classify_news

In [60]:
# Selecting 10 records at a time for inference
prep_news_df2 = prep_news_df.iloc[0:10,:].copy()

In [61]:
# Lambda function to iterate over news articles and save response as a new column
prep_news_df2['predicted_category'] = prep_news_df2['content'].apply(lambda x: predict_news_category(x))

In [62]:
prep_news_df2.to_csv("Instruction_Dataset.csv", index = False)

In [63]:
final_instruction_dataset = pd.read_csv("Instruction_Dataset.csv")

In [66]:
display(final_instruction_dataset[['content', 'predicted_category']])

,content,predicted_category
0,The Democratic Alliance (DA) on Thursday reque...,POLITICS
1,"By Bernadette Carreon AFP\n\nPORT MORESBY, Pap...",ENVIRONMENT
2,SOURCE Wounded Warrior Project\n\nWWP Alumnus ...,Category: IMPACT
3,Italian broadband operator Tiscali has launche...,TECH
4,The board of directors of ServisFirst Bancshar...,BUSINESS
5,PÖYRY PLC Press release 30 September 2015 at...,SCIENCE
6,About EnvironmentGuru \nEnvironmentGuru.com i...,OTHERS
7,Catch Ralph Barrat this Sunday at Simms Park. ...,ARTS & CULTURE
8,Share this page: \n\r\n \t\r \r\n \r\n\r\n\...,BUSINESS
9,"SOURCE Cornerstone Home Lending, Inc.\n\nHOUST...",Category: BUSINESS
